### This notebook implements the connection-oriented cluster match in the paper "Beyond Collusion Resistance: Leveraging Social Information for Plural Funding and Voting" by Joel Miller, E. Glen Weyl, and Leon Erichsen.

In [1]:
import numpy as np    

# !pip install import_ipynb
import import_ipynb

In [2]:
%%capture  
from SBT import *
from SBT_simulation import *

In [3]:
def get_agents_to_clusters_dict_from_clusters_to_agents_dict(clusters_to_agents_dict):
    """
    :param clusters_to_agents_dict: a dictionary mapping a cluster id to the agents in that cluster
    :return: a dictionary mapping a agents to the clusters those agents are a part of
    """

    agents_to_clusters_dict = {}
    for num_agent, agent in enumerate(agents):
        clusters_that_agent_is_in = []
        for cluster, agents_in_cluster in clusters_to_agents_dict.items():
            if agent in agents_in_cluster:
                clusters_that_agent_is_in.append(cluster)
        agents_to_clusters_dict[agent] = clusters_that_agent_is_in
                
    return agents_to_clusters_dict

In [4]:
def get_closeness_matrix(clusters_to_agents_dict, agents_to_clusters_dict):
    """
    :param clusters_to_agents_dict: a dictionary mapping a cluster id to the agents in that cluster
    :param agents_to_clusters_dict: a dictionary mapping a cluster id to the agents in that cluster
    :return: a num_agents x num_agents matrix, where each entry ij is the number of clusters that agent i and agent j have in common
    """
    
    closeness_matrix = np.zeros(shape=(len(agents), len(agents)), dtype=int)    
    for num_agent1, agent1 in enumerate(agents):
        agent1_int = int(agent1)
        for num_agent2, agent2 in enumerate(agents[num_agent1:]):
            agent2_int = int(agent2)
            closeness_matrix[agent1_int, agent2_int] = closeness_matrix[agent2_int, agent1_int] = get_num_common_clusters(agent1, agent2, agents_to_clusters_dict)
            
    return closeness_matrix

In [5]:
def attenuation_modification(agent_contribution):
    """
    :param agent_contribution: the contribution of the agent
    :return: the attenuation modification function applied to the agent's contribution
    """
    
    return np.sqrt(agent_contribution)

In [6]:
def get_attenuation(agent, cluster, clusters_to_agents_dict, agents_to_clusters_dict, closeness_matrix, agents_to_contributions_dict):
    """
    :param agent: the agent for which we want to calculate the attentuation
    :param cluster: the corresponding cluster for which we want to calculate the attentuation
    :param clusters_to_agents_dict: a dictionary mapping a cluster id to the agents in that cluster
    :param agents_to_clusters_dict: a dictionary mapping a cluster id to the agents in that cluster
    :param closeness_matrix: a num_agents x num_agents matrix, where each entry ij is the number of clusters that agent i and agent j have in common
    :param agents_to_contributions_dict: a dictionary mapping an agent id to their contribution
    :return: the attenuation of the agent's contribution
    """
    
    agent_contribution = agents_to_contributions_dict[agent]
    agents_in_cluster = clusters_to_agents_dict[cluster]
    if (agent in agents_in_cluster):
        return attenuation_modification(agent_contribution)
    agent1_int = int(agent)
    for agent_in_cluster in agents_in_cluster:
        agent2_int = int(agent_in_cluster)
        if closeness_matrix[agent1_int, agent2_int] >= 1:
            return attenuation_modification(agent_contribution)        
    else:
        return agent_contribution

In [7]:
def get_num_common_clusters(agent1, agent2, agents_to_clusters_dict):
    """
    :param agent1: the first agent
    :param agent2: the second agent
    :param agents_to_clusters_dict: a dictionary mapping a cluster id to the agents in that cluster
    :return: the number of clusters the two agents have in common
    """
    
    agent1_clusters = agents_to_clusters_dict[agent1]
    agent2_clusters = agents_to_clusters_dict[agent2]
    num_common_clusters = len(set(agent1_clusters).intersection(set(agent2_clusters)))
    
    return num_common_clusters

In [8]:
# initialize variables
clusters_to_agents_dict, agent_budgets, agent_private_beliefs, agent_agent_cluster_affiliation_matrix, cluster_project_affiliation_matrix = initialize_variables()
agents_to_clusters_dict = get_agents_to_clusters_dict_from_clusters_to_agents_dict(clusters_to_agents_dict)
closeness_matrix = get_closeness_matrix(clusters_to_agents_dict, agents_to_clusters_dict)    
# get the agents' modified beliefs (public voices)
agent_private_beliefs_modified_with_project_affiliations = account_for_affiliations(agent_budgets, agent_private_beliefs, agent_agent_cluster_affiliation_matrix, cluster_project_affiliation_matrix)
# create the dictionary mapping projects to dictionaries mapping each agent's contribution to that project
project_to_agents_to_contributions_dict_dict = create_project_to_agents_to_contributions_dict_dict(agent_private_beliefs_modified_with_project_affiliations)
# pick the first project to use in this example
agents_to_contributions_dict = project_to_agents_to_contributions_dict_dict[0]

In [9]:
def connection_oriented_cluster_match(agents_to_contributions_dict, clusters_to_agents_dict):
    """
    :param agents_to_contributions_dict: a dictionary mapping an agent id to their contribution
    :param clusters_to_agents_dict: a dictionary mapping a cluster id to the agents in that cluster
    :return: connection-oriented cluster match
    """
    
    term_on_the_left = sum(agents_to_contributions_dict.values())
    
    # dictionary mapping each agent to the number of clusters that that agent participates in
    agent_to_num_clusters_participating_dict = get_agent_to_num_clusters_participating_dict(clusters_to_agents_dict)
    term_on_the_right = 0
    clusters = list(clusters_to_agents_dict.keys())
    for num_cluster1, cluster1 in enumerate(clusters):
        agents_in_cluster1 = clusters_to_agents_dict[cluster1]
        for num_cluster2, cluster2 in enumerate(clusters[num_cluster1:]):
            agents_in_cluster2 = clusters_to_agents_dict[cluster2]                
                
            first_term = np.sqrt(sum(
                [get_attenuation(agent_in_cluster1, cluster2, clusters_to_agents_dict, agents_to_clusters_dict, closeness_matrix, agents_to_contributions_dict) / 
                 agent_to_num_clusters_participating_dict[agent_in_cluster1] for agent_in_cluster1 in agents_in_cluster1]))
                
            second_term = np.sqrt(sum(
                [get_attenuation(agent_in_cluster2, cluster1, clusters_to_agents_dict, agents_to_clusters_dict, closeness_matrix, agents_to_contributions_dict) / 
                 agent_to_num_clusters_participating_dict[agent_in_cluster2] for agent_in_cluster2 in agents_in_cluster2]))
    
            term_on_the_right += first_term * second_term
        
    return term_on_the_left + term_on_the_right
    

In [10]:
connection_oriented_cluster_match(agents_to_contributions_dict, clusters_to_agents_dict)

2897.434095291871